In [ ]:
from bs4 import BeautifulSoup
import json 
import numpy as np
import pandas as pd
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import urllib
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: / 

In [ ]:
pd.options.display.max_rows = 4000

First we'll use Beautiful soup to parse the wikipedia page

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [ ]:
response = urllib.request.urlopen(url)
rawhtml = response.read()

In [ ]:
soup = BeautifulSoup(rawhtml, 'html.parser')

In [ ]:
pclist = []
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()

        if (b == "Not assigned" and n=="Not assigned"):
            a="skipping"
        elif(n=="Not assigned"):
            pclist.append([pc,b,b])
        else:
            pclist.append([pc,b,n])

In [ ]:
toronto_df = pd.DataFrame(pclist, columns=["FSA", "Borough", "Neighborhood"])
toronto_df = toronto_df.sort_values(by=['FSA'])
toronto_df.head(15)

In [ ]:
toronto_df.shape

create a dataframe out of the retrieved info

In [ ]:
pclist = []
prevPC = ""
currNeighbourhoods = ""
prevBorough = ""
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()
        if ((b == "Not assigned" and n=="Not assigned") or b == "blank"):
            a="skipping"
        else:
            if (n=="Not assigned"):
                n = b;
            if (pc == prevPC):
                currNeighbourhoods = currNeighbourhoods + ", " + n
                 
            else:
                if (prevPC != ""):
                    pclist.append([prevPC,prevBorough,currNeighbourhoods])
                prevPC = pc
                prevBorough = b
                currNeighbourhoods = n 
pclist.append([prevPC,prevBorough,currNeighbourhoods])               

In [ ]:
toronto_df = pd.DataFrame(pclist, columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df = toronto_df.sort_values(by=['PostalCode'])
toronto_df

In [ ]:
toronto_df.shape

Now that we have our data frame, get the latitude and longitude for each postal code and add to the data frame

In [ ]:
!pip install geocoder
import geocoder

In [ ]:
lat_lng_coords = None
lats = []
longs=[]
for index, row in toronto_df.iterrows():
    postal_code=row["PostalCode"]
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    lats.append(lat_lng_coords[0])
    longs.append(lat_lng_coords[1])

In [ ]:
toronto_df['Latitude'] = lats
toronto_df['Longitude'] = longs
toronto_df

In [ ]:
toronto_df.shape

check the size of our df

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[1]
    )
)